In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
import seaborn as sns
import unidecode

In [19]:
#Lendo dados
drogas_final = pd.read_csv('Dados/GESEG_SBX_DROGA_EXPORT.csv', encoding = 'latin-1', sep = ';', index_col=0)
#Pivotando valor "QUANTIDADE" da coluna "ATRIBUTOS" com o mesmo "NRO_INT_OBJETO"
drogas_final_pivot = drogas_final.pivot(index="NRO_INT_OBJETO", columns="ATRIBUTO", values="QUANTIDADE")
#Agrupando dados com o mesmo "K93_NRO_INT_OBJETO"
#Foi preciso utilizar "agg('min')" pois apenas groupby()...
#retorna um objeto iterável 
dg2_final = drogas_final[['NRO_INT_OBJETO','ANO_OCOR', 'NRO_OCOR','NRO_INT_TIPO_OBJETO','TIPO_OBJETO','ATRIBUTO']].groupby(["NRO_INT_OBJETO"]).min()
#Ordenando por "K93_NRO_INT_OBJETO"
dg2_final_sorted = dg2_final.sort_values(by=['NRO_INT_OBJETO'])
drogas_pivot_final_sorted = drogas_final_pivot.sort_values(by=['NRO_INT_OBJETO'])

In [20]:
#Concatenando as duas tabelas
try_1_final = pd.concat([dg2_final_sorted, drogas_pivot_final_sorted],axis=1)
#Atribuindo a tabela resultado "try_1" com um index numérico
resultado = try_1_final.reset_index()
#Renomeando Coluans
resultado = resultado.rename(columns={"NRO_INT_OBJETO": "CHAVE_OBJETO", "TIPO_OBJETO": "TXT_TIPO_OBJETO"})

In [21]:
def ajusta_texto_numerico(texto):
    return str(texto).replace('.', '').replace(',', '.')

resultado['Peso Unitário'] = resultado['Peso Unitário'].apply(ajusta_texto_numerico)
resultado['Peso Total'] = resultado['Peso Total'].apply(ajusta_texto_numerico)

resultado['Peso Unitário'] = pd.to_numeric(resultado['Peso Unitário'], errors='coerce',downcast="float").fillna(0).astype(float)
resultado['Peso Total'] = pd.to_numeric(resultado['Peso Total'], errors='coerce',downcast="float").fillna(0).astype(float)

In [22]:
#Definindo uma series Drogas
#Como resultado do tipo de objeto de resultado = Drogas
Drogas = resultado.loc[resultado['TXT_TIPO_OBJETO'] == 'Drogas']

Drogas = resultado.loc[resultado['TXT_TIPO_OBJETO'] == 'Drogas']
Outras = resultado.loc[resultado['TXT_TIPO_OBJETO'] != 'Drogas']
Outras = Outras.reset_index(drop= True)

#DF com descrição nula
DrogasNA = Drogas.loc[Drogas['Descrição'].isnull()]
DrogasNA = DrogasNA.reset_index(drop=True)

#Excluindo descrição nula do DF Drogas
Drogas = Drogas.loc[Drogas['Descrição'].notna()]
Drogas = Drogas.reset_index(drop=True)

In [23]:
def changer(txt):
    return unidecode.unidecode(str(txt).replace(',','').replace('.',''))

Drogas['Descrição'] = Drogas['Descrição'].apply(changer)
    
Drogas['Descrição'] = Drogas['Descrição'].str.lower()

In [24]:
#Tratando ajustes de drogas
def analizer(x):
    
    if 'maconha' in x:
        return 'Maconha'
    elif 'canabis' in x:
        return 'Maconha'
    elif 'cannabis' in x:
        return 'Maconha'
    elif 'crack' in x:
        return 'Crack'
    elif 'cocaina' in x:
        return 'Cocaína'
    else:
        return 'Drogas'
    
Drogas['RECLASSIFICAÇÃO'] = Drogas['Descrição'].apply(analizer)

In [25]:
listaDrogas = [Drogas, DrogasNA]
Drogas = pd.concat(listaDrogas)

stores_df = Drogas

In [26]:
#Concatenando DF Drogas com DF outros
resultado_final = pd.concat([Drogas, Outras])

#Tratando NaN de reclassificação
resultado_final['RECLASSIFICAÇÃO'] = resultado_final['RECLASSIFICAÇÃO'].fillna(value=resultado_final.TXT_TIPO_OBJETO)

#Definindo a chave do objeto como indice
resultado_final = resultado_final.set_index('CHAVE_OBJETO')

In [27]:
resultado_final

,ANO_OCOR,NRO_OCOR,NRO_INT_TIPO_OBJETO,TXT_TIPO_OBJETO,ATRIBUTO,Cor Predominante,Descrição,Peso Total,Peso Unitário,Quantidade de Unidade(s),Tipo de Embalagem,Unidade de Peso,RECLASSIFICAÇÃO
CHAVE_OBJETO,,,,,,,,,,,,,
19903932,2018,8070,1025,Drogas,Cor Predominante,Verde,uma pequena porcao de erva de cor esverdeada s...,0.0,0.0,1,Porção(es),Miligrama,Maconha
19904961,2018,30422,1025,Drogas,Cor Predominante,Branco,substancia semelhanta a crack,34.0,34.0,1,Porção(es),Grama,Crack
19904971,2018,30422,1025,Drogas,Cor Predominante,Branco,substancia semelhante a cocaina,32.0,32.0,1,Porção(es),Grama,Cocaína
19906420,2018,8084,1025,Drogas,Cor Predominante,Verde,aproximadamente sete gramas,7.0,0.0,6,Tijolo(s),Grama,Drogas
19906441,2018,8084,1025,Drogas,Cor Predominante,Amarelo,aproximadamente sete gramas,7.0,0.0,21,Unidade(s),Grama,Drogas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24126969,2021,2148,5,Maconha,Cor Predominante,Verde,NaN,415.0,0.0,1,Porção(es),Grama,Maconha
24126972,2021,2148,5,Maconha,Cor Predominante,Verde,NaN,5.0,0.0,1,Porção(es),Grama,Maconha
24126984,2021,8335,756,Cocaína,Cor Predominante,Branco,aproximadamente 23 gramas de pinos de cocaína,0.0,0.0,36,Unidade(s),Grama,Cocaína


In [28]:
resultado_final['Unidade de Peso'] = str(resultado_final['Unidade de Peso']) 

In [29]:
def refactor(x,y):
    if 'Miligrama' in x:
        return y/1000,'Miligrama'
    if 'Quilograma' in x:
        return y * 100,'Quilograma'
    if 'Decigrama' in x:
        return y / 10,'Decigrama'
    if 'Tonelada' in x:
        return y * 1000000,'Tonelada'
    else:
        return y,'Grama'
        

#df['newcolumn'] = df.apply(lambda x: fxy(x['A'], x['B']), axis=1)
resultado_final['Peso Unitário Rec','Nova_Unidade de Peso'] = resultado_final.apply(lambda x: refactor(x['Unidade de Peso'], x['Peso Unitário']),axis=1)
#resultado_final['Peso Unitário Rec'] = resultado_final[['Unidade de Peso','Peso Unitário']].apply(refactor)



In [39]:
resultado_final.at[19904961,'Unidade de Peso']

'CHAVE_OBJETO\n19903932    Miligrama\n19904961        Grama\n19904971        Grama\n19906420        Grama\n19906441        Grama\n              ...    \n24126969        Grama\n24126972        Grama\n24126984        Grama\n24127010        Grama\n24127011        Grama\nName: Unidade de Peso, Length: 113231, dtype: object'